# 2 — EDA (mô tả dữ liệu)

EDA chỉ để **mô tả** dữ liệu (shape, missing, phân phối target, histogram/boxplot/correlation…). Không dùng EDA để fit scaler/encoder.

**Phụ thuộc:** đã chạy `app/01_load_clean_split.ipynb` (để có `df`).


## Missing values & target distribution

In [ ]:
# Missing values: đếm số lượng NaN theo từng cột
miss_cnt = df.isna().sum()

# Missing values: tính % NaN theo từng cột
miss_pct = (miss_cnt / len(df) * 100).round(2)

# Tạo bảng tổng hợp missing và sắp xếp giảm dần theo % missing
missing_tbl = (
    pd.DataFrame({"missing_count": miss_cnt, "missing_pct": miss_pct})
      .sort_values("missing_pct", ascending=False)
)

display(missing_tbl)


In [ ]:
# Target distribution: đếm số lượng mỗi lớp (bao gồm cả NaN để kiểm tra dữ liệu bẩn)
y_counts = df["TenYearCHD"].value_counts(dropna=False)

# Target distribution: tính % mỗi lớp
y_pct = (y_counts / y_counts.sum() * 100).round(2)

display(pd.DataFrame({"count": y_counts, "pct": y_pct}))


## Các biểu đồ EDA chính

## EDA (mô tả dữ liệu)
> EDA chỉ để mô tả, **không** dùng để fit scaler/encoder.

In [9]:
# (1) Bảng tổng quan: xem nhanh shape + dtype + số giá trị unique + số missing theo từng cột

# In kích thước dataset (số dòng, số cột)
print("df.shape =", df.shape)

# Tạo bảng overview:
# - dtype: kiểu dữ liệu của cột
# - n_unique: số lượng giá trị khác nhau (kể cả NaN)
# - n_missing: số lượng giá trị bị thiếu
overview = pd.DataFrame({
    "dtype": df.dtypes.astype(str),
    "n_unique": df.nunique(dropna=False),
    "n_missing": df.isna().sum()
}).sort_values("n_missing", ascending=False)  # sắp xếp để cột thiếu nhiều nằm trên

display(overview)


df.shape = (11000, 16)


,dtype,n_unique,n_missing
diaBP,float64,679,412
TenYearCHD,float64,3,361
diabetes,category,3,217
prevalentHyp,category,3,129
age,float64,40,75
BMI,float64,1885,65
gender,category,3,56
heartRate,float64,83,48
cigsPerDay,float64,53,46
totChol,float64,285,40


In [10]:
# (2) Bảng missing: thống kê số lượng (count) và tỷ lệ (%) giá trị thiếu theo từng cột

# Tạo bảng missing gồm:
# - missing_count: tổng số NaN của mỗi cột
# - missing_pct: % NaN của mỗi cột (mean của isna() * 100)
missing_tbl = pd.DataFrame({
    "missing_count": df.isna().sum(),
    "missing_pct": (df.isna().mean() * 100).round(2)
}).sort_values("missing_pct", ascending=False)  # sắp xếp giảm dần theo % missing

display(missing_tbl)


,missing_count,missing_pct
diaBP,412,3.75
TenYearCHD,361,3.28
diabetes,217,1.97
prevalentHyp,129,1.17
age,75,0.68
BMI,65,0.59
gender,56,0.51
heartRate,48,0.44
cigsPerDay,46,0.42
totChol,40,0.36


In [11]:
# (3) Phân phối target (bar + %) — thống kê count/% và vẽ bar chart bằng Plotly

# Ép TenYearCHD về numeric để xử lý trường hợp dữ liệu bị đọc sai kiểu (string, ký tự lạ)
y = pd.to_numeric(df["TenYearCHD"], errors="coerce")

# Loại bỏ giá trị không hợp lệ (inf/-inf) và drop NaN để tránh lỗi khi ép int
y = y.replace([np.inf, -np.inf], np.nan).dropna().astype(int)

# (Tuỳ chọn) Nếu muốn chắc chắn target chỉ là nhị phân 0/1 thì bật dòng dưới
# y = y[y.isin([0, 1])]

# Đếm số lượng mỗi lớp (0/1) và sort theo index để hiển thị theo thứ tự 0 -> 1
counts = y.value_counts().sort_index()

# Tính % theo từng lớp
pct = (counts / counts.sum() * 100).round(2)

# Hiển thị bảng count + percent
display(pd.DataFrame({"count": counts, "percent": pct}))

# Chuẩn bị dataframe cho Plotly (đổi TenYearCHD sang string để trục X hiển thị dạng category)
plot_df = pd.DataFrame({
    "TenYearCHD": counts.index.astype(str),
    "count": counts.values,
    "percent": pct.values
})

# Vẽ bar chart: trục Y là count, label trên cột là percent
fig = px.bar(
    plot_df,
    x="TenYearCHD",
    y="count",
    text="percent",
    title="Target distribution (count + %)"
)

# Format label hiển thị: gắn dấu % và đặt text phía trên cột
fig.update_traces(texttemplate="%{text:.2f}%", textposition="outside")

# Set title trục + tăng y-range để chừa chỗ cho label phía trên
fig.update_layout(
    xaxis_title="TenYearCHD",
    yaxis_title="Count",
    yaxis_range=[0, plot_df["count"].max() * 1.15]
)

fig.show()


,count,percent
TenYearCHD,,
0,9006,84.65
1,1633,15.35


**(4) Histogram cho `age` theo `TenYearCHD` (vẽ gì và đọc như nào?)**

Biểu đồ này đang **so sánh phân phối (distribution)** của biến `age` giữa các nhóm `TenYearCHD`:

- `TenYearCHD = 0` (không mắc CHD trong 10 năm)
- `TenYearCHD = 1` (mắc CHD trong 10 năm)

Cụ thể, nó vẽ **histogram (cột)** của `age` cho từng nhóm và **chồng lên nhau (overlay)** để bạn nhìn xem nhóm nào “lệch” sang tuổi cao hơn/thấp hơn.
Khi bạn dùng `histnorm="probability density"` thì trục Y là **mật độ (density)** → giúp so sánh hình dạng phân phối ngay cả khi 2 nhóm có số lượng khác nhau.

**Vì sao bạn thấy 3 distribution?**
Thường là do `TenYearCHD` có thêm giá trị khác `0/1` (ví dụ: NaN, 2, hoặc string lạ). Cách xử lý là ép về numeric + lọc chỉ giữ {0,1}.


In [12]:
# (4) Histogram/KDE cho age theo target — so sánh phân phối age giữa nhóm CHD=0 và CHD=1

feat = "age"

# Lấy 2 cột cần thiết và loại bỏ missing để plot không bị lỗi
tmp = df[[feat, "TenYearCHD"]].copy()
tmp["TenYearCHD"] = pd.to_numeric(tmp["TenYearCHD"], errors="coerce").replace([np.inf, -np.inf], np.nan)
tmp = tmp.dropna(subset=[feat, "TenYearCHD"])

# Ép target về int rồi đổi sang string để Plotly coi là category khi tô màu
tmp["TenYearCHD"] = tmp["TenYearCHD"].astype(int).astype(str)

if HAS_PLOTLY:
    # Plotly histogram dạng overlay, chuẩn hoá về density để so sánh hình dạng phân phối
    fig = px.histogram(
        tmp,
        x=feat,
        color="TenYearCHD",
        nbins=30,
        barmode="overlay",
        histnorm="probability density",
        title=f"Distribution of {feat} by TenYearCHD"
    )
    fig.update_layout(xaxis_title=feat, yaxis_title="Density")
    fig.show()
else:
    # Fallback Matplotlib: vẽ histogram density cho từng lớp
    fig, ax = plt.subplots()
    for cls in ["0", "1"]:
        data = tmp.loc[tmp["TenYearCHD"] == cls, feat]
        ax.hist(data, bins=30, alpha=0.5, density=True, label=f"CHD={cls}")

    ax.set_title(f"Distribution of {feat} by TenYearCHD")
    ax.set_xlabel(feat)
    ax.set_ylabel("Density")
    ax.legend()
    plt.show()


**(5) Boxplot phát hiện outlier cho các biến số (theo từng lớp TenYearCHD)**

Cell này vẽ **boxplot** cho một nhóm biến numeric quan trọng (BMI, huyết áp, cholesterol, glucose, …) để:

- Quan sát **median** (đường giữa hộp), **IQR** (chiều cao hộp), và mức độ **phân tán** của dữ liệu.
- Phát hiện **outlier** (các điểm nằm ngoài “râu” boxplot).
- So sánh phân phối của từng biến giữa 2 nhóm `TenYearCHD = 0` và `TenYearCHD = 1` thông qua màu sắc.

Cách hoạt động:
- Chọn các cột trong `box_cols` (chỉ lấy những cột thật sự tồn tại).
- Làm sạch `TenYearCHD` để tránh NaN/inf gây lỗi khi ép kiểu.
- Chuyển dữ liệu sang dạng “long format” bằng `melt` để Plotly có thể vẽ nhiều boxplot (mỗi feature là 1 nhóm trên trục X).
- Dùng `points="outliers"` để hiển thị các điểm outlier.


In [13]:
# (5) Boxplot phát hiện outlier (Plotly): so sánh phân phối các biến numeric theo nhóm TenYearCHD

# Danh sách các biến muốn kiểm tra outlier (chỉ lấy cột tồn tại trong df)
box_cols = ["BMI", "sysBP", "diaBP", "totChol", "glucose", "heartRate", "cigsPerDay"]
box_cols = [c for c in box_cols if c in df.columns]

# Lấy subset dữ liệu gồm các biến numeric + target
tmp = df[box_cols + ["TenYearCHD"]].copy()

# Làm sạch target TenYearCHD:
# - ép về numeric để loại bỏ giá trị string/lỗi
# - thay inf/-inf bằng NaN
# - drop các dòng target NaN để tránh lỗi khi ép int
tmp["TenYearCHD"] = pd.to_numeric(tmp["TenYearCHD"], errors="coerce")
tmp["TenYearCHD"] = tmp["TenYearCHD"].replace([np.inf, -np.inf], np.nan)
tmp = tmp.dropna(subset=["TenYearCHD"])

# Ép target về int rồi chuyển sang str để Plotly coi là category khi tô màu
tmp["TenYearCHD"] = tmp["TenYearCHD"].astype(int).astype(str)

# Chuyển dữ liệu sang dạng long: mỗi dòng là (TenYearCHD, feature, value)
# -> Plotly sẽ vẽ boxplot theo feature trên trục X và color theo TenYearCHD
long_df = tmp.melt(id_vars="TenYearCHD", var_name="feature", value_name="value").dropna()

# Vẽ boxplot + hiển thị các điểm outlier
fig = px.box(
    long_df,
    x="feature",
    y="value",
    color="TenYearCHD",
    points="outliers",
    title="Outlier check (boxplot) for key numeric features"
)

# Chỉnh nhãn trục cho gọn
fig.update_layout(xaxis_title="", yaxis_title="Value")
fig.show()


**(6) Heatmap tương quan các biến numeric (Correlation heatmap) — dùng để làm gì?**

Cell này tính và vẽ **ma trận tương quan (correlation matrix)** giữa các biến số (numeric).  
Tương quan (thường là Pearson) nằm trong khoảng **[-1, 1]**:

- **≈ +1**: 2 biến tăng/giảm cùng nhau rất mạnh (tương quan thuận mạnh)
- **≈ -1**: 1 biến tăng thì biến kia giảm mạnh (tương quan nghịch mạnh)
- **≈ 0**: gần như không có quan hệ tuyến tính

**Vì sao bạn thấy “ít feature”?**  
Vì đoạn code chỉ lấy các cột trong list `num_for_corr` (8 cột). Nếu bạn có nhiều feature numeric hơn, bạn cần:
- tự động lấy tất cả numeric columns, hoặc
- mở rộng list `num_for_corr`.

**Dùng bảng này để chọn feature train được không?**  
Dùng **để tham khảo** chứ không nên chọn feature chỉ dựa vào correlation:
- Correlation chỉ đo **quan hệ tuyến tính giữa các feature** (feature-feature), không trực tiếp nói feature nào “dự đoán tốt target”.
- Nó hữu ích để phát hiện **đa cộng tuyến (multicollinearity)**: nếu 2 feature tương quan quá cao (|corr| > 0.85–0.95), mô hình tuyến tính có thể kém ổn định → cân nhắc bỏ bớt 1 feature hoặc dùng regularization (L1/L2).
- Với tree/boosting, đa cộng tuyến thường ít nghiêm trọng hơn, nhưng vẫn có thể gây “trùng thông tin”.


In [14]:
# (6) Heatmap tương quan numeric (Plotly) — hiển thị dạng hình vuông + dễ nhìn hơn

# Cách 1: tự động lấy TẤT CẢ numeric features (khuyến nghị nếu bạn có nhiều feature)
num_for_corr = df.select_dtypes(include=[np.number]).columns.tolist()

# Nếu muốn loại target ra khỏi heatmap (để chỉ xem feature-feature):
# TARGET = "TenYearCHD"
# num_for_corr = [c for c in num_for_corr if c != TARGET]

# Nếu bạn chỉ muốn giữ list cũ thì comment dòng trên và dùng lại list này:
# num_for_corr = ["age","cigsPerDay","totChol","sysBP","diaBP","BMI","heartRate","glucose"]
# num_for_corr = [c for c in num_for_corr if c in df.columns]

# Tính correlation matrix (Pearson mặc định)
corr = df[num_for_corr].corr()

display(corr.round(3))

if HAS_PLOTLY:
    # Plotly heatmap: làm hình vuông bằng cách set width/height bằng nhau
    fig = px.imshow(
        corr,
        text_auto=".2f",          # hiển thị số với 2 chữ số thập phân
        aspect="equal",           # ép ô vuông (square cells)
        title="Correlation heatmap (numeric features)",
        color_continuous_scale="RdBu_r",  # palette dễ nhìn (đỏ-xanh, đảo chiều)
        zmin=-1, zmax=1           # fix thang màu chuẩn cho correlation
    )
    fig.update_layout(width=850, height=850)  # hình vuông
    fig.show()
else:
    import seaborn as sns
    fig, ax = plt.subplots(figsize=(8, 8))
    sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm", square=True, ax=ax)
    ax.set_title("Correlation heatmap (numeric features)")
    plt.show()


,age,education,cigsPerDay,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
age,1.000,-0.143,-0.187,0.242,0.373,0.203,0.128,-0.004,0.100,0.226
education,-0.143,1.000,0.007,-0.011,-0.112,-0.058,-0.122,-0.066,-0.029,-0.055
cigsPerDay,-0.187,0.007,1.000,-0.026,-0.090,-0.058,-0.095,0.067,-0.049,0.045
totChol,0.242,-0.011,-0.026,1.000,0.188,0.159,0.115,0.091,0.034,0.068
sysBP,0.373,-0.112,-0.090,0.188,1.000,0.753,0.322,0.160,0.106,0.202
diaBP,0.203,-0.058,-0.058,0.159,0.753,1.000,0.378,0.164,0.043,0.135
BMI,0.128,-0.122,-0.095,0.115,0.322,0.378,1.000,0.077,0.083,0.077
heartRate,-0.004,-0.066,0.067,0.091,0.160,0.164,0.077,1.000,0.086,0.013
glucose,0.100,-0.029,-0.049,0.034,0.106,0.043,0.083,0.086,1.000,0.116
TenYearCHD,0.226,-0.055,0.045,0.068,0.202,0.135,0.077,0.013,0.116,1.000


**7. Barplot tỷ lệ TenYearCHD theo categorical (%) (3 bảng: gender, currentSmoker, diabetes)**

Mục tiêu của mục (7) là xem **tỷ lệ mắc CHD trong 10 năm (TenYearCHD = 1)** thay đổi như thế nào theo một số biến phân loại quan trọng.

Cách tính trong cả 3 bảng đều giống nhau:
- Lấy 2 cột: biến categorical `c` và `TenYearCHD`
- Làm sạch `TenYearCHD` (ép numeric, bỏ NaN/inf) rồi ép về **0/1**
- Nhóm theo `c` và tính:  
  **CHD rate (%) = mean(TenYearCHD) × 100**  
  (vì `TenYearCHD` là 0/1 nên giá trị trung bình chính là tỷ lệ dương tính)
- Hiển thị **bảng tỷ lệ** và vẽ **bar chart** (trục Y là % CHD)

Ba biểu đồ/bảng tương ứng:
- **7.1**: Tỷ lệ TenYearCHD theo **gender**
- **7.2**: Tỷ lệ TenYearCHD theo **currentSmoker**
- **7.3**: Tỷ lệ TenYearCHD theo **diabetes**

Diễn giải:
- Cột/bar cao hơn nghĩa là nhóm đó có **tỷ lệ CHD cao hơn** trong dữ liệu.
- Nếu thấy xuất hiện “nhóm lạ” hoặc % bất thường, thường do `TenYearCHD` không chỉ có 0/1 hoặc categorical bị lẫn kiểu (Yes/No trộn 0/1) → cần chuẩn hoá dữ liệu trước.


In [15]:
# 7.1) Barplot tỷ lệ TenYearCHD theo gender (%)

c = "gender"
if c in df.columns:
    tmp = df[[c, "TenYearCHD"]].copy()

    tmp["TenYearCHD"] = pd.to_numeric(tmp["TenYearCHD"], errors="coerce").replace([np.inf, -np.inf], np.nan)
    tmp = tmp.dropna(subset=[c, "TenYearCHD"])
    tmp["TenYearCHD"] = tmp["TenYearCHD"].astype(int)

    rate = (tmp.groupby(c)["TenYearCHD"].mean() * 100).reset_index().rename(columns={"TenYearCHD": "chd_rate_pct"})
    display(rate)

    if HAS_PLOTLY:
        fig = px.bar(rate, x=c, y="chd_rate_pct", text="chd_rate_pct", title=f"TenYearCHD rate by {c} (%)")
        fig.update_traces(texttemplate="%{text:.2f}%", textposition="outside")
        fig.update_layout(yaxis_title="CHD rate (%)")
        fig.show()
    else:
        fig, ax = plt.subplots()
        ax.bar(rate[c].astype(str), rate["chd_rate_pct"])
        ax.set_title(f"TenYearCHD rate by {c} (%)")
        ax.set_ylabel("CHD rate (%)"); ax.set_xlabel(c)
        plt.show()
else:
    print("Column 'gender' not found in df.")


,gender,chd_rate_pct
0,female,12.784185
1,male,18.808152


In [16]:
# 7.2) Tỷ lệ TenYearCHD theo currentSmoker (%)

c = "currentSmoker"
if c in df.columns:
    tmp = df[[c, "TenYearCHD"]].copy()

    # Clean target để tránh lỗi + đảm bảo đúng kiểu 0/1
    tmp["TenYearCHD"] = pd.to_numeric(tmp["TenYearCHD"], errors="coerce").replace([np.inf, -np.inf], np.nan)
    tmp = tmp.dropna(subset=[c, "TenYearCHD"])
    tmp["TenYearCHD"] = tmp["TenYearCHD"].astype(int)

    # mean(target) * 100 = % CHD trong từng nhóm currentSmoker
    rate = (tmp.groupby(c, dropna=False)["TenYearCHD"].mean() * 100).reset_index()
    rate = rate.rename(columns={"TenYearCHD": "chd_rate_pct"})
    display(rate)

    if HAS_PLOTLY:
        fig = px.bar(rate, x=c, y="chd_rate_pct", text="chd_rate_pct",
                     title=f"TenYearCHD rate by {c} (%)")
        fig.update_traces(texttemplate="%{text:.2f}%", textposition="outside")
        fig.update_layout(yaxis_title="CHD rate (%)", xaxis_title=c)
        fig.show()
    else:
        fig, ax = plt.subplots()
        ax.bar(rate[c].astype(str), rate["chd_rate_pct"])
        ax.set_title(f"TenYearCHD rate by {c} (%)")
        ax.set_ylabel("CHD rate (%)"); ax.set_xlabel(c)
        plt.show()
else:
    print("Column 'currentSmoker' not found in df.")


,currentSmoker,chd_rate_pct
0,No,14.732143
1,Yes,15.979479


In [17]:
# 7.3) Tỷ lệ TenYearCHD theo diabetes (%)

c = "diabetes"
if c in df.columns:
    tmp = df[[c, "TenYearCHD"]].copy()

    # Clean target để tránh lỗi + đảm bảo đúng kiểu 0/1
    tmp["TenYearCHD"] = pd.to_numeric(tmp["TenYearCHD"], errors="coerce").replace([np.inf, -np.inf], np.nan)
    tmp = tmp.dropna(subset=[c, "TenYearCHD"])
    tmp["TenYearCHD"] = tmp["TenYearCHD"].astype(int)

    # mean(target) * 100 = % CHD trong từng nhóm diabetes
    rate = (tmp.groupby(c, dropna=False)["TenYearCHD"].mean() * 100).reset_index()
    rate = rate.rename(columns={"TenYearCHD": "chd_rate_pct"})
    display(rate)

    if HAS_PLOTLY:
        fig = px.bar(rate, x=c, y="chd_rate_pct", text="chd_rate_pct",
                     title=f"TenYearCHD rate by {c} (%)")
        fig.update_traces(texttemplate="%{text:.2f}%", textposition="outside")
        fig.update_layout(yaxis_title="CHD rate (%)", xaxis_title=c)
        fig.show()
    else:
        fig, ax = plt.subplots()
        ax.bar(rate[c].astype(str), rate["chd_rate_pct"])
        ax.set_title(f"TenYearCHD rate by {c} (%)")
        ax.set_ylabel("CHD rate (%)"); ax.set_xlabel(c)
        plt.show()
else:
    print("Column 'diabetes' not found in df.")


,diabetes,chd_rate_pct
0,No,14.767683
1,Yes,37.086093


**(8) Violinplot theo TenYearCHD — vẽ gì và dùng để làm gì?**

Cell này vẽ **violin plot** để so sánh phân phối của một biến số (mặc định ưu tiên `sysBP`, nếu không có thì dùng `age`) giữa 2 nhóm:

- `TenYearCHD = 0` (không mắc CHD)
- `TenYearCHD = 1` (mắc CHD)

**Violin plot thể hiện gì?**
- Phần “violin” (hình giống cây đàn) cho biết **mật độ phân phối**: chỗ nào phình to nghĩa là nhiều điểm dữ liệu tập trung ở đó.
- Nếu `box=True` (Plotly) / `inner="box"` (Seaborn), bên trong violin có thêm **boxplot**:
  - đường giữa: **median**
  - hộp: **IQR (Q1–Q3)**
  - râu: phạm vi dữ liệu (tuỳ cách tính)
- `points="outliers"` hiển thị các **điểm outlier**.

**Bạn đọc biểu đồ như nào?**
- Nếu nhóm `TenYearCHD=1` có violin/box “dịch lên cao” so với nhóm `0`, có thể biến đó (ví dụ `sysBP`) cao hơn ở nhóm mắc CHD.
- Nhìn độ rộng violin để biết nhóm nào phân tán rộng/hẹp, có nhiều outlier hay không.

Cell có fallback:
- Nếu có Plotly → dùng `px.violin` (đẹp và trực quan).
- Nếu không có Plotly mà có seaborn → `sns.violinplot`.
- Nếu thiếu cả seaborn → fallback sang boxplot (matplotlib).


In [18]:
# (8) Violinplot theo target: so sánh phân phối của 1 feature numeric giữa TenYearCHD=0 và TenYearCHD=1

# Chọn feature để vẽ: ưu tiên sysBP, nếu không có thì dùng age
feat = "sysBP" if "sysBP" in df.columns else ("age" if "age" in df.columns else None)
assert feat is not None, "No suitable numeric feature found for violin/ECDF."

# Lấy dữ liệu cần thiết và loại bỏ missing để plot không lỗi
tmp = df[[feat, "TenYearCHD"]].copy()

# Làm sạch target để tránh lỗi astype(int) do NaN/inf/string lạ
tmp["TenYearCHD"] = pd.to_numeric(tmp["TenYearCHD"], errors="coerce").replace([np.inf, -np.inf], np.nan)
tmp = tmp.dropna(subset=[feat, "TenYearCHD"])

# Ép target về int (0/1) rồi chuyển sang str để coi như category khi plot
tmp["TenYearCHD"] = tmp["TenYearCHD"].astype(int).astype(str)

if HAS_PLOTLY:
    # Plotly violin + box: violin cho hình dạng phân phối, box cho median/IQR, points hiển thị outliers
    fig = px.violin(
        tmp,
        x="TenYearCHD",
        y=feat,
        box=True,
        points="outliers",
        title=f"{feat} by TenYearCHD (violin + box)"
    )
    fig.update_layout(xaxis_title="TenYearCHD", yaxis_title=feat)
    fig.show()

else:
    # Nếu không có Plotly: ưu tiên seaborn violinplot; nếu không có seaborn thì fallback boxplot
    if HAS_SEABORN:
        import seaborn as sns
        fig, ax = plt.subplots()
        sns.violinplot(data=tmp, x="TenYearCHD", y=feat, inner="box", ax=ax)
        ax.set_title(f"{feat} by TenYearCHD (violin)")
        plt.show()
    else:
        fig, ax = plt.subplots()
        ax.boxplot(
            [tmp.loc[tmp["TenYearCHD"] == "0", feat],
             tmp.loc[tmp["TenYearCHD"] == "1", feat]],
            labels=["0", "1"]
        )
        ax.set_title(f"{feat} by TenYearCHD (box fallback)")
        plt.show()
